In [2]:
#import findspark
#findspark.init()
import pyspark
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as func
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("COM6012 Assignment 1 Task2") \
    .getOrCreate()

sc = spark.sparkContext


In [3]:
lines = spark.read.text("ml-25m/ratings.csv").rdd
parts = lines.map(lambda row: row.value.split(","))

header = parts.first()
parts = parts.filter(lambda line: line != header)

In [4]:
for line in parts.take(5):
    print(line)

['1', '296', '5.0', '1147880044']
['1', '306', '3.5', '1147868817']
['1', '307', '5.0', '1147868828']
['1', '665', '5.0', '1147878820']
['1', '899', '3.5', '1147868510']


In [5]:
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)

In [7]:
(training, test) = ratings.randomSplit([0.8, 0.2])

In [9]:
als_1 = ALS(maxIter=10, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

In [11]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")


cv = CrossValidator(estimator = als,
                    evaluator = evaluator,
                    numFolds = 3)

model = cv.fit(training)

KeyError: Param(parent='CrossValidator_40da91e70138a142592e', name='estimatorParamMaps', doc='estimator param maps')

In [ ]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))